## Automate the Boring Stuff with Josh (and Python)

Just as a brief introduction to Python (specifically the Pandas library): here's how you can split comma-separated columns into separate rows. 


### The Use Case
Assume we have some students who've filled out a survey, and they're able to select 1 or many companies they work for and 0 or many occupations they currently have. If they're unemployed they select an "Unemployed" option for the company and don't see the next part of the form asking about their occupation. Easy enough.

Say we've got all that in a .csv file called `sample_data.csv`. If a person has multiple occupations, they have a comma-separated value in the occupation column. That might be fine for basic data retrieval, but it'll get messy once you try to visualize and query the data. 

Recall MIS314: This is a Many-to-Many relationship where a person can have many occupations and an occupation can be had by many persons. What we need is a table that is unique by person (email) and occupation. So someone with two occupations will get two rows. Someone with three will get three rows. You get the idea. 

Once we have that, it'd be very easy to load into a SQL database, produce some visualizations using a Python library of your choice (I prefer [Seaborn](https://seaborn.pydata.org/)) or even move back over to MS Excel if you so desire.  

To do all this, we need to get Python installed on our machine and get the Pandas library installed with it. I'd recommend the [Anaconda distribution](https://www.anaconda.com/products/individual). 


### The Setup
The setup with Anaconda is very straightforward. What I'm creating this in right now is a [Jupyter Notebook](https://jupyter.org/) which allows me to create a document containing markdown (regular text) and code intermingled. Really cool stuff! And Jupyter is included with Anaconda along with a bunch of Python libraries ready-to-go (like Pandas) so you don't have to worry about the installation and configuration hassle!

---

If you have Python and Jupyter installed and aren't sure if you've got Pandas, you can even run shell (CMD) commands via the Jupyter Notebook too!

Running this (a CMD or shell command prefixed with "!"):
```python
!pip install pandas
```

Will install the latest version of Pandas via `pip`, the built-in Python package manager. If you installed Anaconda, you can use `conda`, the Python package manager included with Anaconda. The syntax is exactly the same:

```python
!conda install pandas
```

I usually use `pip` out of habit (and it's fewer keystrokes). 

In [28]:
!pip install pandas

Seems like I've already got it installed! Neat!

Okay let's import the Pandas library into our Jupyter session here. You can always alias your imports using the `as` keyword in Python. 

```Python
import pandas as pd # Aliasing pandas as pd is pretty universal, so I'd recommend doing it
```

I wanted to call something from the Pandas library, it saves me a few extra keystrokes to do `pd.some_thing_i_want()` rather than `pandas.some_thing_i_want()`.

In [29]:
import pandas as pd

You can even import individual modules from a library at a time like:

```Python
from pandas import read_csv as rcsv # If I know I want this module specifically
from pandas import DataFrame as DF # If I know I want this module specifically
```

And now I don't even have to specify `pd.read_csv()` to read in a .csv file. I can just do `rcsv()` because I've already told Python the rcsv alias points to the read_csv module in the Pandas library. 

In [30]:
from pandas import read_csv as rcsv
from pandas import DataFrame as DF

### The Fun Starts Here

Okay so now after a little bit of setup, we're ready to go. 

Let's start by just simply reading in our `sample_data.csv` file using the rcsv module. We'll save this to an object called `data`. 

In [31]:
data = rcsv(filepath_or_buffer='sample_data.csv') # Read the file

print(f'data is a {type(data)}') # Print out a nicely-formatted string showing data's type 

data.head(10) # Print out the first 10 rows of data because I know it's a DataFrame

data is a <class 'pandas.core.frame.DataFrame'>


,email,name,major,company,occupation
0,aepeash@umich.edu,Alannah Peash,MIS,"University of Michigan, University of Michigan","MSI Student, Web Designer Assistant"
1,albert.lewellen@gmail.com,Albert Lewellen,MIS,Urban Science,Data Analyst
2,andersonmikayla@outlook.com,Mikayla Anderson,MIS,"Delphi Technologies, Central Michigan University","Data Analyst, MBA Student"
3,crgrzegorzewski@gmail.com,Clare Grzegorzewski,MIS,KPMG US,Tech. Assurance Assoc.
4,kbozski@gmail.com,Kristiana Bozinovski,MIS,Oakland University,Career Consultant
5,laurenslade17@gmail.com,Lauren Slade,MIS,Quicken Loans,Business Analyst
6,theoneandonlyjoshlinneburg@gmail.com,Josh Linneburg,MIS,IHS Markit,Software Engineer
7,foo@gmail.com,Other Person,Other,Unemployed,NaN


Hey! I know those people!

Okay, so we've got company and occupation and we want to break those out into separate rows. We'll assume the position of the company and occupation is both meaningful and consistent across all rows. 

What I mean: Mikayla is working at Delphi as a Data Analyst and is at CMU as an MBA Student. We'll assume the ordering of the two columns is important and we'll want to make sure our solution honors that original ordering. We don't want her to show up as an MBA Student at Delphi and a Data Analyst at CMU in her two separate rows. 

We've got some Other Person who has no occupation because they're Unemployed. We'll want to make sure we don't lose that record either. 

### The Strategy
So what do we really want to do? I always say the design of a solution is the most important part, not the execution. So what's our design? 
1. Split company out into a separate DataFrame that contains only the email and companies, with emails repeating when someone has >1 companies<br><br>
2. Split occupation out into a separate DataFrame that contains only the email and occupations, with emails repeating when someone has >1 occupations<br><br>
3. Join these two DataFrames back together somehow, preserving the relationship between company and occupation in the original DataFrame (data)<br><br>
4. Join this super-DataFrame with our original DataFrame (data) to get the name and major of that person

There might be more efficient ways to do this that scale better with thousands and millions of rows, but for our purposes that seems reasonable. And it seems like we can easily visualize this to ourselves, right? 

### The Tactics

First thing's first, let's fill in that `NaN` with the string "None" so we don't get any goofiness later on. It's usually much easier to work with data that doesn't have missing values (`NaN` is Pandas/NumPy speak for NULL). 

In [32]:
data = data.fillna('None')
data

,email,name,major,company,occupation
0,aepeash@umich.edu,Alannah Peash,MIS,"University of Michigan, University of Michigan","MSI Student, Web Designer Assistant"
1,albert.lewellen@gmail.com,Albert Lewellen,MIS,Urban Science,Data Analyst
2,andersonmikayla@outlook.com,Mikayla Anderson,MIS,"Delphi Technologies, Central Michigan University","Data Analyst, MBA Student"
3,crgrzegorzewski@gmail.com,Clare Grzegorzewski,MIS,KPMG US,Tech. Assurance Assoc.
4,kbozski@gmail.com,Kristiana Bozinovski,MIS,Oakland University,Career Consultant
5,laurenslade17@gmail.com,Lauren Slade,MIS,Quicken Loans,Business Analyst
6,theoneandonlyjoshlinneburg@gmail.com,Josh Linneburg,MIS,IHS Markit,Software Engineer
7,foo@gmail.com,Other Person,Other,Unemployed,None


Now, let's start #1 and see where we go. For that I'll need to somehow access the company column, I'm thinking. 

I can access columns (which become a Pandas `Series`) with two different forms of notation:

```python
data.company
data['company']
```

Both will work!

In [33]:
foo = data.company
bar = data['company']

if foo.equals(other=bar):
    print('foo and bar equal one another!')
else:
    print('Josh is a liar!')
    
print(f'foo is a {type(foo)}')

foo and bar equal one another!
foo is a <class 'pandas.core.series.Series'>


So what does that look like? 

In [34]:
data.company

0      University of Michigan, University of Michigan
1                                       Urban Science
2    Delphi Technologies, Central Michigan University
3                                             KPMG US
4                                  Oakland University
5                                       Quicken Loans
6                                          IHS Markit
7                                          Unemployed
Name: company, dtype: object

Very cool. Now how do we split this column on ","?

Two ways!

We convert the `Series` to a `str` (string object) using the `str` method (function) of a `Series`.
```python
data.company.str
```

From there we can use the `split()` method (function) of a `str` to split on ",".

```python
data.company.str.split(',')
```

<br>OR<br><br>

We use the `apply()` method along with a `lambda` (anonymous/throwaway) function. This will apply our lambda function to every row in the `Series`. 

Our lambda function will convert each record to a string and will use the `split()` method of a `str` to split on the string ",". 

```python
data.company.apply(lambda x: x.split(','))
```

Something like this also works, but isn't what you'll typically see in Python:
```python
my_lambda_func = lambda x: str(x).split(',')
data.company.apply(my_lambda_func)
```

I like the first way, so let's use that!

In [35]:
data.company.str.split(',')

0    [University of Michigan,  University of Michigan]
1                                      [Urban Science]
2    [Delphi Technologies,  Central Michigan Univer...
3                                            [KPMG US]
4                                 [Oakland University]
5                                      [Quicken Loans]
6                                         [IHS Markit]
7                                         [Unemployed]
Name: company, dtype: object

Interesting stuff! Now we can use the `tolist()` method of a `Series` to get this into a list of lists. 

The name is just like it sounds: The outer list will be our entire column. Each inner list will correspond to a single row in our column. 

In [36]:
companies = data.company.str.split(',').tolist()
companies

[['University of Michigan', ' University of Michigan'],
 ['Urban Science'],
 ['Delphi Technologies', ' Central Michigan University'],
 ['KPMG US'],
 ['Oakland University'],
 ['Quicken Loans'],
 ['IHS Markit'],
 ['Unemployed']]

`companies[0]` returns the first row (Python is 0-indexed) which will be `['University of Michigan', ' University of Michigan']`

`companies[-1]` returns the last row which will be `['Unemployed']`

`companies[0][0]` returns the first item in the first row which will be `'University of Michigan'`

Anyway, let's make this back into a DataFrame...

In [37]:
companies = DF(companies)
companies

,0,1
0,University of Michigan,University of Michigan
1,Urban Science,None
2,Delphi Technologies,Central Michigan University
3,KPMG US,None
4,Oakland University,None
5,Quicken Loans,None
6,IHS Markit,None
7,Unemployed,None


And let's make the index (the unique identifier for each row) the email from the `data` DataFrame.

So when you put it all together, it looks like: 

In [38]:
companies = DF(data.company.str.split(',').tolist(), index=data.email)
companies

,0,1
email,,
aepeash@umich.edu,University of Michigan,University of Michigan
albert.lewellen@gmail.com,Urban Science,None
andersonmikayla@outlook.com,Delphi Technologies,Central Michigan University
crgrzegorzewski@gmail.com,KPMG US,None
kbozski@gmail.com,Oakland University,None
laurenslade17@gmail.com,Quicken Loans,None
theoneandonlyjoshlinneburg@gmail.com,IHS Markit,None
foo@gmail.com,Unemployed,None


Cool enough, but now it's broken out into two separate columns. An improvement, for sure, but not what we want. 

We can "stack" these columns on top of each other using the `stack()` method of a DataFrame.

In [39]:
companies = DF(data.company.str.split(',').tolist(), index=data.email).stack()
companies

email                                  
aepeash@umich.edu                     0          University of Michigan
                                      1          University of Michigan
albert.lewellen@gmail.com             0                   Urban Science
andersonmikayla@outlook.com           0             Delphi Technologies
                                      1     Central Michigan University
crgrzegorzewski@gmail.com             0                         KPMG US
kbozski@gmail.com                     0              Oakland University
laurenslade17@gmail.com               0                   Quicken Loans
theoneandonlyjoshlinneburg@gmail.com  0                      IHS Markit
foo@gmail.com                         0                      Unemployed
dtype: object

Notice how we have email, a number (either 0 or 1) and the company? That 0 or 1 corresponds back to the column from the `companies` dataframe the row is populated from. 

Also notice how where the "1" column is `None`, there's no entry here? It's only Alannah and Mikayla who have more than one entry because they're the only two people with more than one company. Neat!

If we reset the index of the `companies` `Series`, we'll get back a DataFrame. 

In [40]:
companies = companies.reset_index()
companies

,email,level_1,0
0,aepeash@umich.edu,0,University of Michigan
1,aepeash@umich.edu,1,University of Michigan
2,albert.lewellen@gmail.com,0,Urban Science
3,andersonmikayla@outlook.com,0,Delphi Technologies
4,andersonmikayla@outlook.com,1,Central Michigan University
5,crgrzegorzewski@gmail.com,0,KPMG US
6,kbozski@gmail.com,0,Oakland University
7,laurenslade17@gmail.com,0,Quicken Loans
8,theoneandonlyjoshlinneburg@gmail.com,0,IHS Markit
9,foo@gmail.com,0,Unemployed


And now we we're only interested in the "email" and "0" columns. So let's only return those. 

In [41]:
companies = companies.reset_index()[['email', 0]]
companies

,email,0
0,aepeash@umich.edu,University of Michigan
1,aepeash@umich.edu,University of Michigan
2,albert.lewellen@gmail.com,Urban Science
3,andersonmikayla@outlook.com,Delphi Technologies
4,andersonmikayla@outlook.com,Central Michigan University
5,crgrzegorzewski@gmail.com,KPMG US
6,kbozski@gmail.com,Oakland University
7,laurenslade17@gmail.com,Quicken Loans
8,theoneandonlyjoshlinneburg@gmail.com,IHS Markit
9,foo@gmail.com,Unemployed


And we'll make sure "0" gets renamed to "company". 

In [42]:
companies.columns = ['email', 'company']
companies

,email,company
0,aepeash@umich.edu,University of Michigan
1,aepeash@umich.edu,University of Michigan
2,albert.lewellen@gmail.com,Urban Science
3,andersonmikayla@outlook.com,Delphi Technologies
4,andersonmikayla@outlook.com,Central Michigan University
5,crgrzegorzewski@gmail.com,KPMG US
6,kbozski@gmail.com,Oakland University
7,laurenslade17@gmail.com,Quicken Loans
8,theoneandonlyjoshlinneburg@gmail.com,IHS Markit
9,foo@gmail.com,Unemployed


Put that all together in a single cell, and it looks like this:

In [43]:
companies = DF(data.company.str.split(',').tolist(), index=data.email).stack()
companies = companies.reset_index()[['email', 0]]
companies.columns = ['email', 'company']
companies

,email,company
0,aepeash@umich.edu,University of Michigan
1,aepeash@umich.edu,University of Michigan
2,albert.lewellen@gmail.com,Urban Science
3,andersonmikayla@outlook.com,Delphi Technologies
4,andersonmikayla@outlook.com,Central Michigan University
5,crgrzegorzewski@gmail.com,KPMG US
6,kbozski@gmail.com,Oakland University
7,laurenslade17@gmail.com,Quicken Loans
8,theoneandonlyjoshlinneburg@gmail.com,IHS Markit
9,foo@gmail.com,Unemployed


Let's do the exact same thing with occupations:

In [44]:
occupations = DF(data.occupation.str.split(',').tolist(), index=data.email).stack()
occupations = occupations.reset_index()[['email', 0]]
occupations.columns = ['email', 'occupation']
occupations

,email,occupation
0,aepeash@umich.edu,MSI Student
1,aepeash@umich.edu,Web Designer Assistant
2,albert.lewellen@gmail.com,Data Analyst
3,andersonmikayla@outlook.com,Data Analyst
4,andersonmikayla@outlook.com,MBA Student
5,crgrzegorzewski@gmail.com,Tech. Assurance Assoc.
6,kbozski@gmail.com,Career Consultant
7,laurenslade17@gmail.com,Business Analyst
8,theoneandonlyjoshlinneburg@gmail.com,Software Engineer
9,foo@gmail.com,None


Excellent! So now we want to `join` these two DataFrames together using the `join` method. This code is equivalent to the SQL:

```SQL
SELECT 
    c.email
  , c.company
  , o.email as email_r
  , o.occupation
FROM companies c
JOIN occupations o
ON c.rownum = o.rownum -- Because we're technically joining on the index/rownum (0-9) and not the email.
```

In [45]:
company_occupations = companies.join(other=occupations,
                                     rsuffix='_r')

company_occupations

,email,company,email_r,occupation
0,aepeash@umich.edu,University of Michigan,aepeash@umich.edu,MSI Student
1,aepeash@umich.edu,University of Michigan,aepeash@umich.edu,Web Designer Assistant
2,albert.lewellen@gmail.com,Urban Science,albert.lewellen@gmail.com,Data Analyst
3,andersonmikayla@outlook.com,Delphi Technologies,andersonmikayla@outlook.com,Data Analyst
4,andersonmikayla@outlook.com,Central Michigan University,andersonmikayla@outlook.com,MBA Student
5,crgrzegorzewski@gmail.com,KPMG US,crgrzegorzewski@gmail.com,Tech. Assurance Assoc.
6,kbozski@gmail.com,Oakland University,kbozski@gmail.com,Career Consultant
7,laurenslade17@gmail.com,Quicken Loans,laurenslade17@gmail.com,Business Analyst
8,theoneandonlyjoshlinneburg@gmail.com,IHS Markit,theoneandonlyjoshlinneburg@gmail.com,Software Engineer
9,foo@gmail.com,Unemployed,foo@gmail.com,None


And we can drop this extra email column that gets created from the join. 

In [46]:
company_occupations.drop(labels=['email_r'],
                         axis=1,
                         inplace=True)

company_occupations

,email,company,occupation
0,aepeash@umich.edu,University of Michigan,MSI Student
1,aepeash@umich.edu,University of Michigan,Web Designer Assistant
2,albert.lewellen@gmail.com,Urban Science,Data Analyst
3,andersonmikayla@outlook.com,Delphi Technologies,Data Analyst
4,andersonmikayla@outlook.com,Central Michigan University,MBA Student
5,crgrzegorzewski@gmail.com,KPMG US,Tech. Assurance Assoc.
6,kbozski@gmail.com,Oakland University,Career Consultant
7,laurenslade17@gmail.com,Quicken Loans,Business Analyst
8,theoneandonlyjoshlinneburg@gmail.com,IHS Markit,Software Engineer
9,foo@gmail.com,Unemployed,None


### The Finishing Move

Now all we need to do is join this back up with our original DataFrame. 

Two ways to do this:

1. Because `join` always uses the index of the "other" DataFrame as the join condition, if you want to join on, say, email = email, you'll need to set the email in the "other" DataFrame as its index before proceeding. 
<br><br>

2. Using the `merge` method which allows you to join a non-index column to a non-index column, but they have to have the same name in both DataFrames.

In [47]:
# Method 1
data = data[['name' ,'major', 'email']].set_index('email') # Grab the columns we want and set the index
company_occupations.join(other=data, on='email')[['email', 'name', 'major', 'company', 'occupation']] # Join and re-order the columns

,email,name,major,company,occupation
0,aepeash@umich.edu,Alannah Peash,MIS,University of Michigan,MSI Student
1,aepeash@umich.edu,Alannah Peash,MIS,University of Michigan,Web Designer Assistant
2,albert.lewellen@gmail.com,Albert Lewellen,MIS,Urban Science,Data Analyst
3,andersonmikayla@outlook.com,Mikayla Anderson,MIS,Delphi Technologies,Data Analyst
4,andersonmikayla@outlook.com,Mikayla Anderson,MIS,Central Michigan University,MBA Student
5,crgrzegorzewski@gmail.com,Clare Grzegorzewski,MIS,KPMG US,Tech. Assurance Assoc.
6,kbozski@gmail.com,Kristiana Bozinovski,MIS,Oakland University,Career Consultant
7,laurenslade17@gmail.com,Lauren Slade,MIS,Quicken Loans,Business Analyst
8,theoneandonlyjoshlinneburg@gmail.com,Josh Linneburg,MIS,IHS Markit,Software Engineer
9,foo@gmail.com,Other Person,Other,Unemployed,None


In [48]:
# Method 2
data.reset_index(inplace=True) # Reset the index to show Method 2
data = data[['name' ,'major', 'email']] # Grab the columns we want
company_occupations.merge(right=data, on='email', copy=False)[['email', 'name', 'company', 'occupation']] # Merge and re-order the columns

,email,name,company,occupation
0,aepeash@umich.edu,Alannah Peash,University of Michigan,MSI Student
1,aepeash@umich.edu,Alannah Peash,University of Michigan,Web Designer Assistant
2,albert.lewellen@gmail.com,Albert Lewellen,Urban Science,Data Analyst
3,andersonmikayla@outlook.com,Mikayla Anderson,Delphi Technologies,Data Analyst
4,andersonmikayla@outlook.com,Mikayla Anderson,Central Michigan University,MBA Student
5,crgrzegorzewski@gmail.com,Clare Grzegorzewski,KPMG US,Tech. Assurance Assoc.
6,kbozski@gmail.com,Kristiana Bozinovski,Oakland University,Career Consultant
7,laurenslade17@gmail.com,Lauren Slade,Quicken Loans,Business Analyst
8,theoneandonlyjoshlinneburg@gmail.com,Josh Linneburg,IHS Markit,Software Engineer
9,foo@gmail.com,Other Person,Unemployed,None


I like Method #2, so let's use it! I'll also capture the output of each of these steps using the "=" assignment operator. That way the objects actually get updated with the results. This way I can also show you how you can break up this method into an additional step to rearrange the columns (Lines 2 and 3 below were combined into Line 4 above). 

In [49]:
data = data[['name' ,'major', 'email']] # Grab only the columns we want
company_occupations = company_occupations.merge(right=data, on='email', copy=False) # Merge the DF's
company_occupations = company_occupations[['email', 'name', 'company', 'occupation']] # Reorder the col's
company_occupations # Show the result

,email,name,company,occupation
0,aepeash@umich.edu,Alannah Peash,University of Michigan,MSI Student
1,aepeash@umich.edu,Alannah Peash,University of Michigan,Web Designer Assistant
2,albert.lewellen@gmail.com,Albert Lewellen,Urban Science,Data Analyst
3,andersonmikayla@outlook.com,Mikayla Anderson,Delphi Technologies,Data Analyst
4,andersonmikayla@outlook.com,Mikayla Anderson,Central Michigan University,MBA Student
5,crgrzegorzewski@gmail.com,Clare Grzegorzewski,KPMG US,Tech. Assurance Assoc.
6,kbozski@gmail.com,Kristiana Bozinovski,Oakland University,Career Consultant
7,laurenslade17@gmail.com,Lauren Slade,Quicken Loans,Business Analyst
8,theoneandonlyjoshlinneburg@gmail.com,Josh Linneburg,IHS Markit,Software Engineer
9,foo@gmail.com,Other Person,Unemployed,None


Now, say we want to write this out to a .csv in this same working directory? Easy enough!

In [50]:
company_occupations.to_csv(path_or_buf='sample_data_result.csv', index=False)

If you want to see what this looks like as a .csv, you can use the `pprint` module's `pprint` function to nicely print out this comma-separated file!

In [51]:
!pip install pprint
from pprint import pprint
pprint(company_occupations.to_csv(index=False))

('email,name,company,occupation\n'
 'aepeash@umich.edu,Alannah Peash,University of Michigan,MSI Student\n'
 'aepeash@umich.edu,Alannah Peash, University of Michigan, Web Designer '
 'Assistant\n'
 'albert.lewellen@gmail.com,Albert Lewellen,Urban Science,Data Analyst\n'
 'andersonmikayla@outlook.com,Mikayla Anderson,Delphi Technologies,Data '
 'Analyst\n'
 'andersonmikayla@outlook.com,Mikayla Anderson, Central Michigan University, '
 'MBA Student\n'
 'crgrzegorzewski@gmail.com,Clare Grzegorzewski,KPMG US,Tech. Assurance '
 'Assoc.\n'
 'kbozski@gmail.com,Kristiana Bozinovski,Oakland University,Career Consultant\n'
 'laurenslade17@gmail.com,Lauren Slade,Quicken Loans,Business Analyst\n'
 'theoneandonlyjoshlinneburg@gmail.com,Josh Linneburg,IHS Markit,Software '
 'Engineer\n'
 'foo@gmail.com,Other Person,Unemployed,None\n')


And just like that - we're done!

In [52]:
import os
os.system('jupyter nbconvert --to html python_column_splitter.ipynb') # Output this as an HTML

0